In [ ]:
import tensorflow as tf

import numpy as np
import os
import time

In [ ]:
from google.colab import files
uploaded=files.upload()

Saving story.txt to story (2).txt


In [ ]:
# Read, then decode for py2 compat.
text = open('story.txt', 'rb').read().decode(encoding='utf-8')
# length of text is the number of characters in it
print ('Length of text: {} characters'.format(len(text)))

Length of text: 55693 characters


In [ ]:
print(text[:250])

 MY QUEEN: A WEEKLY JOURNAL, OCT 13, 1900 ***




Produced by MWS and the Online Distributed Proofreading
Team at http://www.pgdp.net (This file was produced from
images generously made available by The Internet
Archive/American Libraries.)



In [ ]:
vocab = sorted(set(text))
print ('{} unique characters'.format(len(vocab)))

85 unique characters


In [ ]:
vocab[0:20]

['\n',
 '\r',
 ' ',
 '!',
 '$',
 '&',
 '(',
 ')',
 '*',
 ',',
 '-',
 '.',
 '/',
 '0',
 '1',
 '2',
 '3',
 '4',
 '5',
 '6']

In [ ]:
# Creating a mapping from unique characters to indices
char2idx = {u:i for i, u in enumerate(vocab)}
idx2char = np.array(vocab)

text_as_int = np.array([char2idx[c] for c in text])

In [ ]:
idx2char

array(['\n', '\r', ' ', '!', '$', '&', '(', ')', '*', ',', '-', '.', '/',
       '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '=',
       '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L',
       'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'Y', '_',
       'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm',
       'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z',
       'é', 'œ', '—', '‘', '’', '“', '”'], dtype='<U1')

In [ ]:
char2idx

{'\n': 0,
 '\r': 1,
 ' ': 2,
 '!': 3,
 '$': 4,
 '&': 5,
 '(': 6,
 ')': 7,
 '*': 8,
 ',': 9,
 '-': 10,
 '.': 11,
 '/': 12,
 '0': 13,
 '1': 14,
 '2': 15,
 '3': 16,
 '4': 17,
 '5': 18,
 '6': 19,
 '7': 20,
 '8': 21,
 '9': 22,
 ':': 23,
 ';': 24,
 '=': 25,
 '?': 26,
 'A': 27,
 'B': 28,
 'C': 29,
 'D': 30,
 'E': 31,
 'F': 32,
 'G': 33,
 'H': 34,
 'I': 35,
 'J': 36,
 'K': 37,
 'L': 38,
 'M': 39,
 'N': 40,
 'O': 41,
 'P': 42,
 'Q': 43,
 'R': 44,
 'S': 45,
 'T': 46,
 'U': 47,
 'V': 48,
 'W': 49,
 'Y': 50,
 '_': 51,
 'a': 52,
 'b': 53,
 'c': 54,
 'd': 55,
 'e': 56,
 'f': 57,
 'g': 58,
 'h': 59,
 'i': 60,
 'j': 61,
 'k': 62,
 'l': 63,
 'm': 64,
 'n': 65,
 'o': 66,
 'p': 67,
 'q': 68,
 'r': 69,
 's': 70,
 't': 71,
 'u': 72,
 'v': 73,
 'w': 74,
 'x': 75,
 'y': 76,
 'z': 77,
 'é': 78,
 'œ': 79,
 '—': 80,
 '‘': 81,
 '’': 82,
 '“': 83,
 '”': 84}

In [ ]:
text_as_int[0:10]

array([ 2, 39, 50,  2, 43, 47, 31, 31, 40, 23])

In [ ]:
# The maximum length sentence we want for a single input in characters
seq_length = 100
examples_per_epoch = len(text)//(seq_length+1)

# Create training examples / targets
char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)

for i in char_dataset.take(10):
  print(idx2char[i.numpy()])

 
M
Y
 
Q
U
E
E
N
:


In [ ]:
sequences = char_dataset.batch(seq_length+1, drop_remainder=True)

for item in sequences.take(5):
  print(repr(''.join(idx2char[item.numpy()])))

' MY QUEEN: A WEEKLY JOURNAL, OCT 13, 1900 ***\r\n\r\n\r\n\r\n\r\nProduced by MWS and the Online Distributed Pro'
'ofreading\r\nTeam at http://www.pgdp.net (This file was produced from\r\nimages generously made available'
' by The Internet\r\nArchive/American Libraries.)\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\nMY QUEEN\r\n\r\nA WEEKLY JOURNAL FOR YO'
'UNG WOMEN\r\n\r\nNo. 3. PRICE, FIVE CENTS.\r\n\r\nMARION MARLOWE’S TRUE HEART\r\n\r\nOR\r\n\r\nHOW A DAUGHTER FORGAVE'
'\r\n\r\nBY GRACE SHIRLEY\r\n\r\nPUBLISHED WEEKLY BY STREET & SMITH, 238 William Street, New York City.\r\n\r\n_Co'


In [ ]:
def split_input_target(chunk):
  input_text = chunk[:-1]
  target_text = chunk[1:]
  return input_text, target_text

dataset = sequences.map(split_input_target)

In [ ]:
for input_example, target_example in  dataset.take(1):
  print ('Input data: ', repr(''.join(idx2char[input_example.numpy()])))
  print ('Target data:', repr(''.join(idx2char[target_example.numpy()])))

Input data:  ' MY QUEEN: A WEEKLY JOURNAL, OCT 13, 1900 ***\r\n\r\n\r\n\r\n\r\nProduced by MWS and the Online Distributed Pr'
Target data: 'MY QUEEN: A WEEKLY JOURNAL, OCT 13, 1900 ***\r\n\r\n\r\n\r\n\r\nProduced by MWS and the Online Distributed Pro'


In [ ]:
# Batch size
BATCH_SIZE = 64

# Buffer size to shuffle the dataset
# (TF data is designed to work with possibly infinite sequences,
# so it doesn't attempt to shuffle the entire sequence in memory. Instead,
# it maintains a buffer in which it shuffles elements).
BUFFER_SIZE = 10000

dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

dataset

<BatchDataset shapes: ((64, 100), (64, 100)), types: (tf.int64, tf.int64)>

In [ ]:
# Length of the vocabulary in chars
vocab_size = len(vocab)

# The embedding dimension
embedding_dim = 256

# Number of RNN units
rnn_units = 1024

#drop out  rate
drop_rate=0.1

second change -adding activation layer as tanh in LSTM
third change-adding kernel initializer as glorot_uniforms to LSTM layer
fourth chnage-adding dropout layer with 0.1 drop rate
adding activation layer to dense layer


In [ ]:
def build_model(vocab_size, embedding_dim, rnn_units, batch_size,drop_rate):
  model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim,
                              batch_input_shape=[batch_size, None]),
    tf.keras.layers.LSTM(rnn_units,
                        activation='sigmoid',
                        return_sequences=True,
                        kernel_initializer='glorot_uniform',
                        stateful=True,
                        recurrent_initializer='glorot_uniform'),
    tf.keras.layers.Dropout(drop_rate),                    
    tf.keras.layers.Dense(vocab_size)
  ])
  return model

In [ ]:
model = build_model(
    vocab_size = len(vocab),
    embedding_dim=embedding_dim,
    rnn_units=rnn_units,
    batch_size=BATCH_SIZE,
    drop_rate = drop_rate)

In [ ]:
for input_example_batch, target_example_batch in dataset.take(1):
  example_batch_predictions = model(input_example_batch)
  print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")

(64, 100, 85) # (batch_size, sequence_length, vocab_size)


In [ ]:
model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (64, None, 256)           21760     
_________________________________________________________________
lstm_1 (LSTM)                (64, None, 1024)          5246976   
_________________________________________________________________
dropout_3 (Dropout)          (64, None, 1024)          0         
_________________________________________________________________
dense_3 (Dense)              (64, None, 85)            87125     
Total params: 5,355,861
Trainable params: 5,355,861
Non-trainable params: 0
_________________________________________________________________


In [ ]:
sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)
sampled_indices = tf.squeeze(sampled_indices,axis=-1).numpy()
#This gives us, at each timestep, a prediction of the next character index:
sampled_indices

array([44, 35, 26, 49, 49,  7, 73, 78, 45, 13,  7, 40, 40, 39, 68, 48, 52,
       48, 83, 39, 66, 75, 33, 18, 55, 60, 78, 39, 69, 84, 44,  0, 75, 31,
       49, 29, 53, 36, 19, 76, 77, 46, 46, 83, 28, 68, 36,  4, 23, 68, 64,
       11, 73, 41, 24, 29, 69, 15, 69, 51, 70, 71, 77, 24, 71, 74, 77, 77,
       63, 50, 46,  3, 81, 59, 40, 55, 52, 25, 29, 57, 36, 18, 30,  5, 27,
       56, 59, 49, 25, 82, 62,  8,  2, 38, 71, 25, 27, 73, 17, 62])

In [ ]:
def loss(labels, logits):
  return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

example_batch_loss  = loss(target_example_batch, example_batch_predictions)
print("Prediction shape: ", example_batch_predictions.shape, " # (batch_size, sequence_length, vocab_size)")
print("scalar_loss:      ", example_batch_loss.numpy().mean())

Prediction shape:  (64, 100, 85)  # (batch_size, sequence_length, vocab_size)
scalar_loss:       4.44377


In [ ]:
model.compile(optimizer='adam', loss=loss)

In [ ]:
# Directory where the checkpoints will be saved
checkpoint_dir = './training_checkpoints'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback=tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

In [ ]:
EPOCHS=20
history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])

Epoch 1/20
8/8 [==============================] - 4s 322ms/step - loss: 3.4893
Epoch 2/20
8/8 [==============================] - 3s 317ms/step - loss: 3.2167
Epoch 3/20
8/8 [==============================] - 3s 312ms/step - loss: 3.1537
Epoch 4/20
8/8 [==============================] - 3s 312ms/step - loss: 3.0960
Epoch 5/20
8/8 [==============================] - 3s 315ms/step - loss: 3.0190
Epoch 6/20
8/8 [==============================] - 3s 318ms/step - loss: 2.9224
Epoch 7/20
8/8 [==============================] - 3s 322ms/step - loss: 2.8230
Epoch 8/20
8/8 [==============================] - 3s 308ms/step - loss: 2.7176
Epoch 9/20
8/8 [==============================] - 3s 317ms/step - loss: 2.6379
Epoch 10/20
8/8 [==============================] - 3s 323ms/step - loss: 2.5815
Epoch 11/20
8/8 [==============================] - 3s 314ms/step - loss: 2.5383
Epoch 12/20
8/8 [==============================] - 3s 322ms/step - loss: 2.4971
Epoch 13/20
8/8 [==============================] 

In [ ]:
tf.train.latest_checkpoint(checkpoint_dir)
model = build_model(vocab_size, embedding_dim, rnn_units, batch_size=1, drop_rate=drop_rate)

model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))

model.build(tf.TensorShape([1, None]))
model.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_4 (Embedding)      (1, None, 256)            21760     
_________________________________________________________________
lstm_2 (LSTM)                (1, None, 1024)           5246976   
_________________________________________________________________
dropout_4 (Dropout)          (1, None, 1024)           0         
_________________________________________________________________
dense_4 (Dense)              (1, None, 85)             87125     
Total params: 5,355,861
Trainable params: 5,355,861
Non-trainable params: 0
_________________________________________________________________


In [ ]:
def generate_text(model, start_string):
  # Evaluation step (generating text using the learned model)

  # Number of characters to generate
  num_generate = 1000

  # Converting our start string to numbers (vectorizing)
  input_eval = [char2idx[s] for s in start_string]
  input_eval = tf.expand_dims(input_eval, 0)

  # Empty string to store our results
  text_generated = []

  # Low temperatures results in more predictable text.
  # Higher temperatures results in more surprising text.
  # Experiment to find the best setting.
  temperature = 1.0

  # Here batch size == 1
  model.reset_states()
  for i in range(num_generate):
    predictions = model(input_eval)
    # remove the batch dimension
    predictions = tf.squeeze(predictions, 0)

    # using a categorical distribution to predict the character returned by the model
    predictions = predictions / temperature
    predicted_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()

    # We pass the predicted character as the next input to the model
    # along with the previous hidden state
    input_eval = tf.expand_dims([predicted_id], 0)

    text_generated.append(idx2char[predicted_id])

  return (start_string + ''.join(text_generated))

In [ ]:
print(generate_text(model, start_string=u"Queen: "))

Queen: I Yoned
g ne srel ser ofatine cite vem ift sher,
hand son as aidte shevea sasplowie whalis be s1diigr. leste an tuin welisg ylive pinssulin ter bell he tousy ald at fMarove nit orceby a’s.  hara toe ifer, ufean, shic tny tiithe Itauld. Shea tein a dower S onay oul, ass ant thus alde, Mat o delly t thinr naint avee noullg ried Istrean!” RC—aDk
he, wrouc wof
he hat  we le  haw that
Mfun af tris acmass haplond at!”

tuus arsald therh, her weorsRaf ma ppulr hiy tiplowitses, Maisheander dou, iog jes ancthe ratthe hep thie
asuss the at iluy
 jf ans yim hece

I wher man sed an’sande.


“E—Tull tor yar,” arlisicre nof sed llio frey Sae queve.
now athacead. ind wit, wery threo hanncessyous Nos yve rhu
sit afcey Thalsse ine an tountees af out futo la


Ars bitcede allywyous ismee f waf ink, dather ts fot drirl fe to woone stoll, andad abo pove maves gheveelyen y arithneas ondqunew when sas orlyo Jo titest
Agrams fertestitr yo murly yomed see kevewed shede a tins ce celd apan 
